In [1]:
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

In [2]:
class BasicLSTM(nn.Module):
    def __init__(self, i_size, o_size, h_size, emb_size,
                 n_layers=1, dropout_p=0.5, use_cuda=False):
        super(BasicLSTM, self).__init__()
        print("Building Basic LSTM model...")
        self.n_layers = n_layers
        self.h_size = h_size
        self.use_cuda = use_cuda

        self.embedding = nn.Embedding(o_size, emb_size)
        self.dropout = nn.Dropout(dropout_p)
        self.lstm = nn.LSTM(emb_size, h_size, num_layers=n_layers)
        self.out = nn.Linear(h_size, o_size)

    def forward(self, inputs, targets=None, state=None):
        state = self._init_state(b_size=inputs.size()[1])

        # inputs: [i, b]
        embedded = self.embedding(inputs)  # [i, b, e]
        out, _ = self.lstm(embedded, state)  # [i, b, h]
        decoded = self.out(out[-1])  # [b, h] -> [b, o]
        logits = F.softmax(decoded)
        return logits

    def _init_state(self, b_size=1):
        weight = next(self.parameters()).data
        return (
            Variable(weight.new(self.n_layers, b_size, self.h_size).zero_()),
            Variable(weight.new(self.n_layers, b_size, self.h_size).zero_())
        )